# Building a Smart Knowledge Base with OpenAI Assistants File Search

## Introduction

In this tutorial, we'll learn how to use the File Search capability of OpenAI's Assistants API to create a smart knowledge base system. We'll build a customer support assistant that can answer questions about product documentation, user guides, and FAQs.

File Search allows Assistants to access knowledge from documents provided by you or your users. OpenAI automatically handles:
- Parsing and chunking documents
- Creating and storing embeddings
- Performing vector and keyword search
- Retrieving relevant content to answer queries

## Setup

First, let's set up our environment with the required imports and initialize our OpenAI client:

In [ ]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"var: ")

_set_env("OPENAI_API_KEY")

In [1]:
from openai import OpenAI
import time

# Initialize the OpenAI client
client = OpenAI()

In [2]:
import os
from typing_extensions import override
from openai import AssistantEventHandler

# Create a helper function to print responses nicely
def print_response(message):
    if hasattr(message.content[0], 'text'):
        print(f"{message.role}: {message.content[0].text.value}")

## Creating the Knowledge Base Assistant

Let's create an assistant specifically designed to handle product documentation queries:

In [3]:
# Create the assistant with file search enabled
assistant = client.beta.assistants.create(
    name="Product Support Assistant",
    instructions="""You are a helpful product support assistant. 
    Use the provided documentation to answer customer questions accurately.
    If you're not sure about something, admit it and stick to the information in the documents.""",
    model="gpt-4o",
    tools=[{"type": "file_search"}]
)

print(f"Created assistant with ID: {assistant.id}")

Created assistant with ID: asst_JdEjTHdrxGRTHSopdn8F8uxM


## Setting Up the Vector Store

The File Search tool uses Vector Store objects to manage and search through your documents. Let's create one for our product documentation:

In [4]:
# Create a vector store for our documentation
vector_store = client.beta.vector_stores.create(
    name="Product Documentation"
)

print(f"Created vector store with ID: {vector_store.id}")

# Update the assistant to use our vector store
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={
        "file_search": {
            "vector_store_ids": [vector_store.id]
        }
    }
)

Created vector store with ID: vs_68752c0f952481918ba7105fb2ea5c28


## Adding Documentation Files

Now let's create a function to add files to our vector store. We'll use the upload_and_poll helper to ensure files are processed completely:

In [5]:
def add_files_to_vector_store(file_paths, vector_store_id):
    """
    Upload files to the vector store and wait for processing to complete
    """
    # Prepare the files for upload
    file_streams = [open(path, "rb") for path in file_paths]
    
    try:
        # Upload files and wait for processing
        file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
            vector_store_id=vector_store_id,
            files=file_streams
        )
        
        print(f"Upload status: {file_batch.status}")
        print(f"File counts: {file_batch.file_counts}")
        return file_batch
        
    finally:
        # Clean up file streams
        for stream in file_streams:
            stream.close()

## Creating a Thread Manager

Let's create a class to manage our customer support threads:

In [6]:
class SupportThreadManager:
    def __init__(self, assistant_id):
        self.client = OpenAI()
        self.assistant_id = assistant_id
        
    def create_thread(self):
        """Create a new support thread"""
        self.thread = self.client.beta.threads.create()
        return self.thread
    
    def add_message(self, content, file_id=None):
        """Add a message to the thread"""
        message_params = {
            "thread_id": self.thread.id,
            "role": "user",
            "content": content
        }
        
        if file_id:
            message_params["attachments"] = [{
                "file_id": file_id,
                "tools": [{"type": "file_search"}]
            }]
            
        return self.client.beta.threads.messages.create(**message_params)
    
    def process_annotations(self, message):
        """Process message annotations to format citations"""
        if not message.content or not message.content[0].text:
            return "No text content found"
        
        message_content = message.content[0].text
        annotations = message_content.annotations
        citations = []
        
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(
                annotation.text, 
                f' [{index}]'
            )
            
            if hasattr(annotation, 'file_citation'):
                cited_file = self.client.files.retrieve(annotation.file_citation.file_id)
                citations.append(
                    f'[{index}] {annotation.file_citation.quote} from {cited_file.filename}'
                )
                
        final_content = message_content.value
        if citations:
            final_content += '\n\nSources:\n' + '\n'.join(citations)
            
        return final_content
    
    def get_response(self, event_handler=None):
        """Run the assistant and get a response"""
        with self.client.beta.threads.runs.stream(
            thread_id=self.thread.id,
            assistant_id=self.assistant_id,
            event_handler=event_handler,
        ) as stream:
            stream.until_done()
            
        # Get the latest message
        messages = self.client.beta.threads.messages.list(
            thread_id=self.thread.id,
            order="desc",
            limit=1
        )
        
        latest_message = messages.data[0]
        return self.process_annotations(latest_message)

## Creating an Event Handler

To handle streaming responses:

In [7]:
class SupportEventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)
    
    @override
    def on_text_delta(self, delta, snapshot):
        print(delta.value, end="", flush=True)
    
    @override
    def on_tool_call_created(self, tool_call):
        print(f"\nsearching documentation...\n", flush=True)

## Putting It All Together

Let's create a complete example of using our support system:

In [8]:
# Initialize our support thread manager
support_manager = SupportThreadManager(assistant.id)

# Create a new thread
thread = support_manager.create_thread()

# Example customer query
query = "How do I reset my password?"
support_manager.add_message(query)

# Get and print the response
response = support_manager.get_response(SupportEventHandler())
print("\nFormatted Response:")
print(response)


searching documentation...


assistant > It seems the document you uploaded does not contain specific instructions on how to reset your password. If you're trying to reset your password for a particular service or platform, typically, you would go to the login page and select an option like "Forgot Password?" or "Reset Password" and follow the instructions provided. If you have a specific service in mind, let me know so I can provide more tailored guidance!
Formatted Response:
It seems the document you uploaded does not contain specific instructions on how to reset your password. If you're trying to reset your password for a particular service or platform, typically, you would go to the login page and select an option like "Forgot Password?" or "Reset Password" and follow the instructions provided. If you have a specific service in mind, let me know so I can provide more tailored guidance!


## Additional Features: Thread-Specific Documents

Sometimes customers might provide additional context in the form of documents. Here's how to handle that:

In [9]:
def handle_customer_attachment(file_path, thread_manager):
    """Handle a customer-provided document"""
    # Upload the file
    file = client.files.create(
        file=open(file_path, "rb"),
        purpose="assistants"
    )
    
    # Add message with attachment
    thread_manager.add_message(
        "Please refer to the attached document for context.",
        file_id=file.id
    )
    
    return file.id

## Best Practices and Tips

1. **Document Preparation**:
   - Keep documents under 512 MB
   - Use supported file formats (PDF, DOCX, TXT, etc.)
   - Structure documents clearly with headers and sections

2. **Vector Store Management**:
   - Create separate vector stores for different types of documentation
   - Monitor the status of file processing before running queries
   - Clean up or archive outdated documentation

3. **Query Handling**:
   - Be specific in the assistant's instructions
   - Process and display citations properly
   - Handle errors gracefully

## Clean Up Function

Here's a utility function to clean up resources:

In [ ]:
def cleanup_resources(vector_store_id=None, file_ids=None):
    """Clean up vector stores and files when no longer needed"""
    if vector_store_id:
        try:
            client.beta.vector_stores.delete(vector_store_id)
            print(f"Deleted vector store: {vector_store_id}")
        except Exception as e:
            print(f"Error deleting vector store: {e}")
    
    if file_ids:
        for file_id in file_ids:
            try:
                client.files.delete(file_id)
                print(f"Deleted file: {file_id}")
            except Exception as e:
                print(f"Error deleting file: {e}")

## Conclusion

In this tutorial, we've built a comprehensive support system using OpenAI's Assistants API with File Search. The system can:
- Manage documentation in vector stores
- Handle customer queries with context from documentation
- Process customer-provided documents
- Provide cited responses from the knowledge base

Try experimenting with different types of documentation and queries to see how the system performs in various scenarios. Remember to handle your API keys securely and manage your resources efficiently in production environments.